<br><font size=6><b>Efficient <font color=#383838><img style="display: inline;margin-top:0px;padding-top:50px;margin-left:-12px;" src="Data/yelp-logo.png" width="14%"/>API</font></b></font><font size=6> Calls</font>



<font size=3><ul><li>Suganthan Thamotharampillai
<br><li>01.31.2023
<div style="border-style:solid;border-color:red;border-width:10px;border-width:0px;border-top-width:7px;">

In [ ]:
z